Algoritmos

In [1]:
# @title Record
#Abstract Record
class Record:
  def __init__(self, record_size, value=None):
    self.byte_array = bytearray([0] * record_size)
    self.write(value)

  # creates a record from a bytearray
  def create(record_size, byte_array):
   raise NotImplementedError("Subclasses must implement create")

  def write(self, value):
    raise NotImplementedError("Subclasses must implement write")

  def read(self):
   raise NotImplementedError("Subclasses must implement read")

  def size(self):
    return len(self.byte_array)

  def bytes(self):
    return self.byte_array.copy()

  def __str__(self):
    return f"value:{self.read()}, capacity:{self.size()}"

In [3]:
# @title DataEntry
import struct
class DataEntry(Record):
  #productId: char(11), title: char(33), userId: char(22), profileName: char(33), helpfulness: char(6), score: float(4), time: Integer (4), summary: char(33), text: char(121), total=267
  def __init__(self, productId, title, userId, profileName, helpfulness, score, time, summary, text):
    self.byte_array = bytearray([0] * 267)

    DataEntry.__checkType(productId, str)
    DataEntry.__checkType(title, str)
    DataEntry.__checkType(userId, str)
    DataEntry.__checkType(profileName, str)
    DataEntry.__checkType(helpfulness, str)
    DataEntry.__checkType(score, float)
    DataEntry.__checkType(time, int)
    DataEntry.__checkType(summary, str)
    DataEntry.__checkType(text, str)

    productId = productId.encode('utf-8')[:10]
    title = title.encode('utf-8')[:32]
    userId = userId.encode('utf-8')[:21]
    profileName = profileName.encode('utf-8')[:32]
    helpfulness = helpfulness.encode('utf-8')[:5]
    summary = summary.encode('utf-8')[:32]
    text = text.encode('utf-8')[:120]

    self.write(productId, title, userId, profileName, helpfulness, score, time, summary, text)

  def create(byte_array):
    (productId, title, userId, profileName, helpfulness, score, time, summary, text) = DataEntry.__unpack(byte_array)
    return DataEntry(productId, title, userId, profileName, helpfulness, score, time, summary, text)

  def __checkType(var, type):
    if(not isinstance(var, type)):
       raise TypeError(f"The '{var}' must be {type}!")

  def __writeRec(byte_array, pos, value, type, size):
    if(type=="int"):
      byte_array[pos:pos+4] = struct.pack('I', value)
    elif(type=="str"):
      byte_array[pos] = len(value)
      byte_array[pos+1:pos+1+len(value)] = value
    elif(type=="float"):
      byte_array[pos:pos+4] = struct.pack('f', value)
    else:
      raise TypeError(f"Type must be 'int', 'float' or 'str'!")
    return pos+size

  def write(self, productId, title, userId, profileName, helpfulness, score, time, summary, text):
    #productId: char(10)
    pos = DataEntry.__writeRec(self.byte_array, 0, productId, "str", 11)

    #title: char(33)
    pos = DataEntry.__writeRec(self.byte_array, pos, title, "str", 33)

    #userId: char(22)
    pos = DataEntry.__writeRec(self.byte_array, pos, userId, "str", 22)

    #profileName: char(33)
    pos = DataEntry.__writeRec(self.byte_array, pos, profileName, "str", 33)

    #helpfulness: char(6)
    pos = DataEntry.__writeRec(self.byte_array, pos, helpfulness, "str", 6)

    #score: float(4)
    pos = DataEntry.__writeRec(self.byte_array, pos, score, "float", 4)

    #time: Integer (4)
    pos = DataEntry.__writeRec(self.byte_array, pos, time, "int", 4)

    #summary: char(33)
    pos = DataEntry.__writeRec(self.byte_array, pos, summary, "str", 33)

    #text: char(121)
    DataEntry.__writeRec(self.byte_array, pos, text, "str", 121)

  def read(self):
    return DataEntry.__unpack(self.byte_array)

  def __unpack(byte_array):
    pos = 0

    productId = byte_array[pos+1:pos+1+byte_array[pos]].decode('utf-8')
    pos = pos+11

    title = byte_array[pos+1:pos+1+byte_array[pos]].decode('utf-8')
    pos = pos+33

    userId = byte_array[pos+1:pos+1+byte_array[pos]].decode('utf-8')
    pos = pos+22

    profileName = byte_array[pos+1:pos+1+byte_array[pos]].decode('utf-8')
    pos = pos+33

    helpfulness = byte_array[pos+1:pos+1+byte_array[pos]].decode('utf-8')
    pos = pos+6

    score = struct.unpack('f', byte_array[pos:pos+4])[0]
    pos = pos+4

    time = struct.unpack('I', byte_array[pos:pos+4])[0]
    pos = pos+4

    summary = byte_array[pos+1:pos+1+byte_array[pos]].decode('utf-8')
    pos = pos+33

    text = byte_array[pos+1:pos+1+byte_array[pos]].decode('utf-8')

    return (productId, title, userId, profileName, helpfulness, score, time, summary, text)

In [4]:
# @title DataEntry test
rec = DataEntry("Id-3", "Title-3", "UserId-3", "ProfileName-3", "9/10", 3.0, 940636400, "summary-3", "text-2")
print(rec)
print(f"Bytes: {rec.bytes()}")
rec2 = DataEntry.create(rec.bytes())
print(rec2)

value:('Id-3', 'Title-3', 'UserId-3', 'ProfileName-3', '9/10', 3.0, 940636400, 'summary-3', 'text-2'), capacity:267
Bytes: bytearray(b'\x04Id-3\x00\x00\x00\x00\x00\x00\x07Title-3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08UserId-3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\rProfileName-3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x049/10\x00\x00\x00@@\xf0\xf8\x108\tsummary-3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06text-2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [5]:
# @title Block
import struct

class Block:
  def __init__(self, block_size, record_size):
    self.records = []
    self.block_size = block_size
    self.record_size = record_size
    self.capacity = (block_size-2) // record_size #two bytes are used to store the number or records

  # creates a block from a bytearray
  def create(block_size, record_size, byte_array, Record):
    block = Block(block_size, record_size)

    num_records = struct.unpack('>H', byte_array[:2])[0]
    pos = 2
    for _ in range(num_records):
      rec = Record.create(byte_array[pos:pos+record_size] )
      block.add(rec)
      pos += record_size

    return block

  def add(self, record):
    if(self.size() < self.capacity):
      self.records.append(record)
    else:
      raise ValueError("The block is full!")

  def addIndex(self, index, record):
    if(self.size() < self.capacity):
      self.records.insert(index, record)
    else:
      raise ValueError("The block is full!")

  def remove(self, keyPos, keyValue):
    if(not isinstance(keyPos, int)):
       raise TypeError(f"The keyPos must be an int!")

    rec = self.search(keyPos, keyValue)
    if(rec):
      return self.records.remove(rec)
    return None

  def removeIndex(self, index):
    if(not isinstance(index, int)):
       raise TypeError(f"The index must be int!")
    return self.records.pop(index)

  def removeLast(self):
    if len(self.records)>0:
      return self.records.pop()
    return None

  def read(self):
    str=""
    for rec in self.records:
      str += f"{rec.read()}\n"
    return str;

  #returns the position of the record within the block
  def search(self, keyPos, keyValue):
    if(not isinstance(keyPos, int)):
       raise TypeError(f"The key must be an int!")

    for i in range(len(self.records)):
      if(self.getRecord(i).read()[keyPos]==keyValue):
        return i
    return -1

  # returns the list of records, within the range keyA (inclusive) and KeyB (exclusive)
  def rangeSearch(self, keyPos, keyValueA, keyValueB):
    if(not isinstance(keyPos, int)):
       raise TypeError(f"The keyPos must be an int!")

    ret = []
    for rec in self.records:
      recValue = rec.read()[keyPos]
      if(recValue>=keyValueA and recValue<keyValueB):
        ret.append(rec)
    return ret

  def getRecord(self, index):
    return self.records[index]

  def getFirtRecord(self):
    return self.records[0]

  def getLastRecord(self):
    return self.records[-1]

  def size(self):
    return len(self.records)

  def isFull(self):
    return self.size()==self.capacity

  def isEmpty(self):
    return self.size()==0

  def bytes(self):
    byte_array = bytearray([0] * self.block_size)

    byte_array[:2] = struct.pack('>H', self.size()) #pack int into two bytes
    pos = 2
    for rec in self.records:
      byte_array[pos:pos+rec.size()] = rec.bytes()
      pos += rec.size()

    return byte_array

  
      

In [6]:
dataBlock = Block(4096, 267)
print(f"block capacity: {dataBlock.capacity}")
print(f"block size: {dataBlock.size()}")

dataBlock.add(DataEntry("Id-1", "Title-1", "UserId-1", "ProfileName-1", "5/7", 5.0, 940636700, "summary-1", "text-1"))
dataBlock.add(DataEntry("Id-2", "Title-2", "UserId-2", "ProfileName-2", "6/8", 4.0, 940636600, "summary-2", "text-2"))
dataBlock.add(DataEntry("Id-3", "Title-3", "UserId-3", "ProfileName-3", "9/10", 3.0, 940636400, "summary-3", "text-2"))


print(dataBlock.read())

print(f"getRecord(1): {dataBlock.getRecord(1)}")
print(f"search(1,'Title-2'):{dataBlock.search(1,'Title-2')}")
print(f"range search(5, 2.0, 4.5):")
for rec in dataBlock.rangeSearch(5, 2.0, 4.5):
  print(rec)
print(f"block bytes: {dataBlock.bytes()}")

block2 = Block.create(4096, 267, dataBlock.bytes(), DataEntry)
print(block2.read())

block capacity: 15
block size: 0
('Id-1', 'Title-1', 'UserId-1', 'ProfileName-1', '5/7', 5.0, 940636700, 'summary-1', 'text-1')
('Id-2', 'Title-2', 'UserId-2', 'ProfileName-2', '6/8', 4.0, 940636600, 'summary-2', 'text-2')
('Id-3', 'Title-3', 'UserId-3', 'ProfileName-3', '9/10', 3.0, 940636400, 'summary-3', 'text-2')

getRecord(1): value:('Id-2', 'Title-2', 'UserId-2', 'ProfileName-2', '6/8', 4.0, 940636600, 'summary-2', 'text-2'), capacity:267
search(1,'Title-2'):1
range search(5, 2.0, 4.5):
value:('Id-2', 'Title-2', 'UserId-2', 'ProfileName-2', '6/8', 4.0, 940636600, 'summary-2', 'text-2'), capacity:267
value:('Id-3', 'Title-3', 'UserId-3', 'ProfileName-3', '9/10', 3.0, 940636400, 'summary-3', 'text-2'), capacity:267
block bytes: bytearray(b'\x00\x03\x04Id-1\x00\x00\x00\x00\x00\x00\x07Title-1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08UserId-1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\rProfileName-1\x00\x00\x00\x00

In [7]:
# @title HeapFile
import os
class HeapFile:

  def __init__(self, filename, block_size, record_size, Record, recordKeyId, create=False):
    self.filename = filename
    self.block_size = block_size
    self.record_size = record_size
    self.Record = Record
    self.keyPos = recordKeyId

    #print(f"os.path.abspath(filename): {os.path.abspath(filename)}")

    if(create and os.path.exists(filename)):
      os.remove(filename)

    if((not create) and os.path.exists(filename)):
      self.blocks = os.path.getsize(filename)//block_size
    else:
      if(os.path.exists(filename)):
        os.remove(filename)

      self.blocks = 1
      with open(self.filename, "wb") as file:
        file.write(Block(block_size, record_size).bytes())

  def add(self, record):
    block = self.read(self.blocks-1)

    if(block.isFull()):
      self.blocks+=1
      block = Block(self.block_size, self.record_size)

    block.add(record)
    self.write(self.blocks-1, block)

  def addNewBlock(self):
    block = Block(self.block_size, self.record_size)
    self.write(self.blocks, block)
    self.blocks+=1

  def remove(self, key):
    (block_id, rec_id) = self.search(key)
    if block_id>=0:
      block = self.read(block_id)
      block.removeIndex(rec_id)
      self.write(block_id, block)

  def scan(self, output=True):
    for i in range(self.blocks):
      if output:
        print(self.read(i).read())
      else:
        self.read(i).read()

  # returns the record, containing the key
  def search(self, key):
    for i in range(self.blocks):
      block = self.read(i)
      rec_id = block.search(self.keyPos, key)

      if(rec_id>=0):
        return (i, rec_id)
    return (-1, -1)

  # returns a list of records, within the range keyA (inclusive) and KeyB (exclusive)
  def rangeSearch(self, keyA, keyB):
    result = []
    for i in range(self.blocks):
      result += self.read(i).rangeSearch(self.keyPos, keyA, keyB)
    return result

  def write(self, block_id, block):
    if(not isinstance(block, Block)):
       raise TypeError(f"The block must by of Block type!")

    with open(self.filename, "r+b") as file:
      file.seek(self.block_size * (block_id))
      file.write(block.bytes())


  def readFirst(self):
    return self.read(0)

  def readLast(self):
    return self.read(self.blocks-1)

  def read(self, block_id):
    with open(self.filename, "r+b") as file:
      file.seek(self.block_size *(block_id))
      byte_array = file.read(self.block_size)
      return Block.create(self.block_size, self.record_size, byte_array, self.Record)



In [8]:
# @title HeapFile test
filename  = "heapfile.bin"
fileType = HeapFile(filename, 600, 267, DataEntry, 1)
fileType.add(DataEntry("Id-1", "Title-1", "UserId-1", "ProfileName-1", "5/7", 5.0, 940636700, "summary-1", "text-1"))
fileType.add(DataEntry("Id-2", "Title-2", "UserId-2", "ProfileName-2", "6/8", 4.0, 940636600, "summary-2", "text-2"))
fileType.add(DataEntry("Id-3", "Title-3", "UserId-3", "ProfileName-3", "9/10", 3.0, 940636400, "summary-3", "text-3"))
fileType.add(DataEntry("Id-4", "Title-4", "UserId-4", "ProfileName-4", "7/8", 2.0, 940636300, "summary-4", "text-4"))
fileType.add(DataEntry("Id-5", "Title-5", "UserId-5", "ProfileName-5", "2/4", 3.5, 940636200, "summary-5", "text-5"))

fileType.scan()

print(f"blocks:{fileType.blocks}")

block1 = fileType.readFirst()
print(f"block1.getRecord(0)={block1.getRecord(1)}")

block2 = fileType.readLast()
print(f"block2.getRecord(1)={block2.getRecord(0)}")

print(f"range search(1, Title-2, Title-5):")
for rec in fileType.rangeSearch("Title-2", "Title-5"):
  print(rec)

os.remove(filename)

('1882931173', 'Its Only Art If Its Well Hung!', 'AVCGYZL8FQQTD', 'Jim of Oz "jim-of-oz"', '7/7', 4.0, 940636800, 'Nice collection of Julie Strain ', "This is only for Julie Strain fans. It's a collection of her photos -- about 80 pages worth with a nice section of paint")
('0826414346', 'Dr. Seuss: American Icon', 'A30TK6U7DNS82R', 'Kevin Killian', '10/10', 5.0, 1095724800, 'Really Enjoyed It', "I don't care much for Dr. Seuss but after reading Philip Nel's book I changed my mind--that's a good testimonial to the ")

('0826414346', 'Dr. Seuss: American Icon', 'A3UH4UZ4RSVO82', 'John Granger', '10/11', 5.0, 1078790400, 'Essential for every personal and', 'If people become the books they read and if "the child is father to the man," then Dr. Seuss (Theodor Seuss Geisel) is t')
('0826414346', 'Dr. Seuss: American Icon', 'A2MVUWT453QH61', 'Roy E. Perry "amateur philosophe', '7/7', 4.0, 1090713600, 'Phlip Nel gives silly Seuss a se', 'Theodore Seuss Geisel (1904-1991), aka &quot;Dr. Seuss,

In [9]:
#productId: char(11), title: char(33), userId: char(22), profileName: char(33), helpfulness: char(6), score: float(4), time: Integer (4), summary: char(33), text: char(121), total=267

def parser(fileType: HeapFile):
    dataFileName = "data/Sample.txt"
    
    if(os.path.exists(dataFileName)):        
        with open(dataFileName, "r") as inputFile:
            while True:
                line = None
                               
                dataSample = {}
                for i in range(10):
                    line = inputFile.readline()
                    if not line:
                        break
                  
                    
                    key, value = line.split(':', 1)

                    if key != 'product/price':
                        dataSample[key] = value.strip()
         
                if not line:
                        break     

                line = inputFile.readline()

                if not line:
                        break 

                # print("\n")
                dataEntry = DataEntry(dataSample['product/productId'],
                                      dataSample['product/title'],
                                      dataSample['review/userId'],
                                      dataSample['review/profileName'],
                                      dataSample['review/helpfulness'],
                                      float(dataSample['review/score']),
                                      int(dataSample['review/time']),
                                      dataSample['review/summary'],
                                      dataSample['review/text'])
                                      
                                      
                
                # print(dataEntry)
                fileType.add(dataEntry)


                    
                
              

In [11]:
filename  = "heapfile.bin"
fileType = HeapFile(filename, 600, 267, DataEntry, 1)
parser(fileType)

# print(f"Size in blocks: {fileType.blocks}")

In [12]:
filename  = "heapfile.bin"
fileType = HeapFile(filename, 600, 267, DataEntry, 1)
fileType.scan()

('1882931173', 'Its Only Art If Its Well Hung!', 'AVCGYZL8FQQTD', 'Jim of Oz "jim-of-oz"', '7/7', 4.0, 940636800, 'Nice collection of Julie Strain ', "This is only for Julie Strain fans. It's a collection of her photos -- about 80 pages worth with a nice section of paint")
('0826414346', 'Dr. Seuss: American Icon', 'A30TK6U7DNS82R', 'Kevin Killian', '10/10', 5.0, 1095724800, 'Really Enjoyed It', "I don't care much for Dr. Seuss but after reading Philip Nel's book I changed my mind--that's a good testimonial to the ")

('0826414346', 'Dr. Seuss: American Icon', 'A3UH4UZ4RSVO82', 'John Granger', '10/11', 5.0, 1078790400, 'Essential for every personal and', 'If people become the books they read and if "the child is father to the man," then Dr. Seuss (Theodor Seuss Geisel) is t')
('0826414346', 'Dr. Seuss: American Icon', 'A2MVUWT453QH61', 'Roy E. Perry "amateur philosophe', '7/7', 4.0, 1090713600, 'Phlip Nel gives silly Seuss a se', 'Theodore Seuss Geisel (1904-1991), aka &quot;Dr. Seuss,

In [13]:
print(f"Size in blocks: {fileType.blocks}")

Size in blocks: 454


In [14]:
block_size = 600
record_size = 267

In [15]:
def swap(block: Block, i: int, j: int):
    i_temp = block.getRecord(i)
    j_temp = block.getRecord(j)
    block.records.remove(block.getRecord(i))    
    block.addIndex(i, j_temp)
    block.records.remove(block.getRecord(j))
    block.addIndex(j, i_temp)

In [16]:
def sort_block(block: Block, keyPos):
    for i in range(block.size() - 1, 0, -1):
        for j in range(i):
            if block.getRecord(j + 1).read()[keyPos] < block.getRecord(j).read()[keyPos]:
                swap(block, j, j + 1)

In [17]:
def clear_block(block):
    # print('Cleaning Block')
    while not block.isEmpty():
        block.removeLast()
        # print(f"Block size: {block.size()}")

In [161]:
def intercalate(blocks, keyPos):
    for block in blocks:
        sort_block(block, keyPos)
        
    for i in range(len(blocks)):
        for j in range(i + 1, len(blocks)):
            minimum = {'block': 0, 'record': 0}
            permute = {'block': 0, 'record': 0}
            needsPermute = False
            # print(f"i : {i}, j:{j}")
            for k in range(blocks[i].size()):
                for l in range(blocks[j].size()):
                    # print(f"Block: {i}, key: {blocks[i].getRecord(k).read()[keyPos]} and Block: {j}, key: {blocks[j].getRecord(l).read()[keyPos]} ")
                    if blocks[i].getRecord(k).read()[keyPos] > blocks[j].getRecord(l).read()[keyPos]:
                        # print(f"Will permute Block: {i}, key: {blocks[i].getRecord(k).read()[keyPos]} and Block: {j}, key: {blocks[j].getRecord(l).read()[keyPos]} ")
                        # print(f"Will permute Block: {i}, record: {k} and Block: {j}, record: {l} ")
                        minimum['block'] = j
                        minimum['record'] =l

                        permute['block'] = i
                        permute['record'] = k
                        needsPermute = True
                if needsPermute:
                    # print(f"Minimum: {minimum}")
                    # print(f"Permute: {permute}")
                    record1 = blocks[minimum['block']].getRecord(minimum['record'])
                    
                    record2 = blocks[permute['block']].getRecord(permute['record'])

                    blocks[minimum['block']].removeIndex(minimum['record'])
                    blocks[minimum['block']].addIndex(minimum['record'], record2)

                    blocks[permute['block']].removeIndex(permute['record'])
                    blocks[permute['block']].addIndex(permute['record'], record1)
                    needsPermute = False
                    
    for block in blocks:
        sort_block(block, keyPos)

                    
                    
            

In [162]:
keyPos = 5
b = 3
filename  = "heapfile.bin"
fileType = HeapFile(filename, 600, 267, DataEntry, 0)

blocks = []
for i in range( 15, 15 + b):
    blocks.append(fileType.read(i))

for i in range(len(blocks)):
    print(f"Printing block: {i}")
    for j in range(blocks[i].size()):
        print(blocks[i].getRecord(j).read()[keyPos])

intercalate(blocks, keyPos)

print('\nAfter intercalate:\n')
for i in range(len(blocks)):
    print(f"Printing block: {i}")
    for j in range(blocks[i].size()):
        print(blocks[i].getRecord(j).read()[keyPos])


Printing block: 0
5.0
5.0
Printing block: 1
4.0
5.0
Printing block: 2
4.0
5.0

After intercalate:

Printing block: 0
4.0
4.0
Printing block: 1
5.0
5.0
Printing block: 2
5.0
5.0


In [160]:
import math
import os.path

block_byte_size = 600
record_byte_size = 267

def external_sort(fileType: HeapFile, keyPos: int, b = 3):
    numberOfSeries = math.ceil(fileType.blocks / b)
    seriesFiles = []

    # Cria um buffer capaz de acomodar b páginas
    inbuffer = Block(block_byte_size * b, record_byte_size)

    startPos = 0
    endPos = b
    """ Passagem 0: Cria N/B séries ordenadas, com B páginas cada, 
        onde N é o total de páginas"""
    for i in range(numberOfSeries):
        if endPos > fileType.blocks:
            endPos = fileType.blocks
            
        for j in range(startPos, endPos):
            block = fileType.read(j)
            for k in range(block.size()):
                record = block.getRecord(k)
                inbuffer.add(record)

                if inbuffer.isFull():
                    sort_block(inbuffer, keyPos )
                    filename  = f"walk_0_series_{i}.bin"
                    seriesFile = HeapFile(filename, 600, 267, DataEntry, 1)
                    
                    for l in range(inbuffer.size()):
                        seriesFile.add(inbuffer.getRecord(l))
                        
                    clear_block(inbuffer)
        startPos = endPos
        endPos = endPos + b
        
        """É executado, caso o buffer tenha lido as últimas páginas 
        e não esteja cheio"""
        if (not inbuffer.isEmpty()) and (not inbuffer.isFull()):
            sort_block(inbuffer, keyPos)
            filename  = f"walk_0_series_{i}.bin"
            seriesFile = HeapFile(filename, 600, 267, DataEntry, 1)
            
            for l in range(inbuffer.size()):
                        seriesFile.add(inbuffer.getRecord(l))
            # seriesFile.write(0, inbuffer)
        clear_block(inbuffer)


    
    
    buffer = []
    """ Passagens 1 a log b N """
    # Cria um buffer para acomodar (b - 1) páginas 
    # buffer = Block(block_byte_size * (b - 1), record_byte_size)
    for i in range(b -1):
        block = Block(block_byte_size, record_byte_size)
        buffer.append(block)
    
    # Buffer de saída
    outputBuffer = Block(block_byte_size, record_byte_size)    

    # Contagem de passagens
    walk = 0    

    seriesSize = b   
    
    outputWalk = 0

    newNumberOfSeries = numberOfSeries
    newSeriesSize = seriesSize    
    
    while numberOfSeries > 1:
        print(f"Walk: {walk}")
        print(f"Number of Series: {numberOfSeries}")
        print(f"Series Size: {seriesSize}")          
        
        # Quantidade de séries processadas por passagem
        processedSeries = 0

        newSeriesSize = (b - 1) * seriesSize
        newNumberOfSeries = math.ceil(numberOfSeries / (b - 1)) 
        
        outputWalk = outputWalk + 1
        startSeries = 0
        endSeries = b - 1          
        
        while processedSeries < numberOfSeries:            
            newSeriesFiles = []                

            for i in range(newNumberOfSeries):
                filename = f"walk_{outputWalk}_series_{i}.bin" 
                series = HeapFile(filename, block_byte_size, record_byte_size, DataEntry, 1)
                newSeriesFiles.append(series)
            
            for k in range(len(newSeriesFiles)):  
                totalSize = 0
                pageIndex = 0
                seriesFileIndex = 0
                seriesFiles = []
                
                # Abre as próximas b - 1 séries 
                for i in range(startSeries, endSeries):
                    filename = f"walk_{walk}_series_{i}.bin"
                    if os.path.isfile(filename):
                        series = HeapFile(filename, block_byte_size, record_byte_size, DataEntry, 0)
                        seriesFiles.append(series)
                        totalSize = totalSize + series.blocks          

                fileLength = 0                   
                processedSeries = processedSeries + len(seriesFiles)
                canIntercalate = True
                contentCount = 0
                
                while fileLength < totalSize:
                    bufferOk = True

                    # print(f"File Length: {fileLength}, Total Size: {totalSize}")
                    for l in range(b - 1):
                        while not buffer[l].isFull():   
                            if pageIndex < seriesFiles[seriesFileIndex].blocks:
                                # print(f"Page Index: {pageIndex}, File Size: {seriesFiles[seriesFileIndex].blocks}")
                                block = seriesFiles[seriesFileIndex].read(pageIndex) 
                                buffer[l] = block                                                  
    
                                seriesFileIndex = seriesFileIndex + 1
                                if seriesFileIndex == (len(seriesFiles)):
                                    seriesFileIndex = 0
                                    pageIndex = pageIndex + 1
                                    
                            if pageIndex == seriesFiles[seriesFileIndex].blocks and not buffer[l].isFull():                              
                                bufferOk = True
                                break
                
                            else:
                                for block in buffer:
                                    if not block.isFull():
                                        bufferOk = False
                        if bufferOk:
                            break

                    # if not buffer[0].isEmpty():    
                                           
                    intercalate(buffer, keyPos)

                    outputBuffer = buffer[0]
                    newSeriesFiles[k].write(fileLength, outputBuffer)
                   
                    fileLength = fileLength + 1

                    # Limpa o buffer de saída
                    clear_block(outputBuffer)

                    # limpa a primeira página do buffer para receber uma nova
                    clear_block(buffer[0])
                           
            
                startSeries = endSeries
               
                # processedSeries = endSeries
                endSeries = endSeries + (b - 1)
                
                # print(f"End Series: {endSeries}")
                # print(f"Processed Series: {processedSeries}")
            
                
        walk = outputWalk
        numberOfSeries = newNumberOfSeries
        seriesSize = newSeriesSize
                                 
            
   

In [149]:
keyPos = 5
b = 5
filename  = "heapfile.bin"
fileType = HeapFile(filename, 600, 267, DataEntry, 0)
print(fileType.blocks)
external_sort(fileType, keyPos, b)

454
Walk: 0
Number of Series: 114
Series Size: 4
Walk: 1
Number of Series: 38
Series Size: 12
Walk: 2
Number of Series: 13
Series Size: 36
Walk: 3
Number of Series: 5
Series Size: 108
Walk: 4
Number of Series: 2
Series Size: 324


In [150]:
seriesFile0 = HeapFile('walk_4_series_0.bin', 600, 267, DataEntry, 0)
print(seriesFile0.blocks)
seriesFile0.scan()


324
('0595344550', 'Whispers of the Wicked Saints', 'A3Q12RK71N74LB', 'Book Reader', '7/11', 1.0, 1117065600, 'not good', 'I bought this book because I read some glowing praise on an online library site. Unfortunately, I was deeply disappointe')
('0595344550', 'Whispers of the Wicked Saints', 'AUR0VA5H0C66C', 'LoveToRead "Actually Read Books"', '1/2', 1.0, 1119225600, 'Buyer beware', 'This is a self-published book, and if you want to know why--read a few paragraphs! Those 5 star reviews must have been w')

('076192177X', 'The Handbook of Community Practi', 'A2T39WC4SVQ3NQ', 'K. T. Pehrson', '0/2', 4.0, 1254009600, 'textbook', 'Book shipped quickly and was in excellent condition as stated. Easy transaction would buy again')
('006000486X', 'Tess and the Highlander', 'AVWFMN5CELC8Q', 'sarah', '6/6', 4.0, 1082592000, 'a great book for Historical roma', 'This is an engaging a count of life of Tess a girl who at a young age washed up on the shore of the Isle of May. With no')

('0671551345',

In [131]:
print('Scanning Series 0')
seriesFile0 = HeapFile('passage_0_series_0.bin', 600, 267, DataEntry, 0)
seriesFile0.scan()

print('Scanning Series 150')
seriesFile1 = HeapFile('passage_0_series_150.bin', 600, 267, DataEntry, 0)
seriesFile1.scan()



Scanning Series 0
('1882931173', 'Its Only Art If Its Well Hung!', 'AVCGYZL8FQQTD', 'Jim of Oz "jim-of-oz"', '7/7', 4.0, 940636800, 'Nice collection of Julie Strain ', "This is only for Julie Strain fans. It's a collection of her photos -- about 80 pages worth with a nice section of paint")
('0826414346', 'Dr. Seuss: American Icon', 'A2MVUWT453QH61', 'Roy E. Perry "amateur philosophe', '7/7', 4.0, 1090713600, 'Phlip Nel gives silly Seuss a se', 'Theodore Seuss Geisel (1904-1991), aka &quot;Dr. Seuss,&quot; was one of the most influential writers and artists of the')

('0826414346', 'Dr. Seuss: American Icon', 'A22X4XUPKF66MR', 'D. H. Richards "ninthwavestore"', '3/3', 4.0, 1107993600, 'Good academic overview', 'Philip Nel - Dr. Seuss: American IconThis is basically an academic overview of Seuss poetry, art, cartoons, and the prob')
('0826414346', 'Dr. Seuss: American Icon', 'A2F6NONFUDB6UK', 'Malvin', '2/2', 4.0, 1127174400, "One of America's greatest creati", '"Dr. Seuss: American Ico

In [95]:
block = fileType.read(2)
keyPos = 3
# print(block.size())
sort_block(block, keyPos, 0, block.size() - 1)
for rec in block.records:
    print(rec.read()[keyPos])

2/2
3/3


In [104]:
external_sort(fileType, 4)

In [105]:
filename  = "heapfile.bin"
fileType = HeapFile(filename, 600, 267, DataEntry, 0)
fileType.scan()

('0826414346', 'Dr. Seuss: American Icon', 'A30TK6U7DNS82R', 'Kevin Killian', '10/10', 5.0, 1095724800, 'Really Enjoyed It', "I don't care much for Dr. Seuss but after reading Philip Nel's book I changed my mind--that's a good testimonial to the ")
('1882931173', 'Its Only Art If Its Well Hung!', 'AVCGYZL8FQQTD', 'Jim of Oz "jim-of-oz"', '7/7', 4.0, 940636800, 'Nice collection of Julie Strain ', "This is only for Julie Strain fans. It's a collection of her photos -- about 80 pages worth with a nice section of paint")

('0826414346', 'Dr. Seuss: American Icon', 'A3UH4UZ4RSVO82', 'John Granger', '10/11', 5.0, 1078790400, 'Essential for every personal and', 'If people become the books they read and if "the child is father to the man," then Dr. Seuss (Theodor Seuss Geisel) is t')
('0826414346', 'Dr. Seuss: American Icon', 'A2MVUWT453QH61', 'Roy E. Perry "amateur philosophe', '7/7', 4.0, 1090713600, 'Phlip Nel gives silly Seuss a se', 'Theodore Seuss Geisel (1904-1991), aka &quot;Dr. Seuss,

In [103]:
'T' < 'm'

True